<a href="https://colab.research.google.com/github/AfsarAI/ML-Assignment-3/blob/main/IITM_MLP_KA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mlp_term_2_2025_kaggle_assignment_3_path = kagglehub.competition_download('mlp-term-2-2025-kaggle-assignment-3')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#!pip install sentence_transformers

In [ ]:
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sentence_transformers import SentenceTransformer

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import (
    LogisticRegression, RidgeClassifier,
    PassiveAggressiveClassifier, Perceptron
)
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
df = pd.read_csv('/kaggle/input/mlp-term-2-2025-kaggle-assignment-3/train.csv')
test_df = pd.read_csv('/kaggle/input/mlp-term-2-2025-kaggle-assignment-3/test.csv')

# Identify data types of different columns

In [ ]:
df.info()

Drop Columns

In [ ]:
for i in (df, test_df):
  i.drop(columns=['id'], inplace=True)

In [ ]:
for i in (df, test_df):
  i.drop(columns=['store_name', 'category'], inplace=True)

Converting review_type to numeric

In [ ]:
def get_hours_ago(time: str) -> int:
  components = time.split(' ')
  num_value = components[0]
  if num_value == 'a':
    num_value = 1
  else:
    num_value = int(num_value)

  if components[1].startswith('hour'):
    multiplier = 1
  elif components[1].startswith('day'):
    multiplier = 24
  elif components[1].startswith('week'):
    multiplier = 24 * 7
  elif components[1].startswith('month'):
    multiplier = 24 * 30
  elif components[1].startswith('year'):
    multiplier = 24 * 365

  return num_value * multiplier

for i in (df, test_df):
  i['review_time'] = i['review_time'].apply(get_hours_ago)
  i.rename(columns={'review_time': 'hours_ago'}, inplace=True)

Removing Space

In [ ]:
df.rename(columns={'latitude ': 'latitude'}, inplace=True)

Removing comma

In [ ]:
for i in (df, test_df):
  i['rating_count'] = i['rating_count'].str.replace(',', '').astype(int)

Dropping unimportant

In [ ]:
for i in (df, test_df):
  i.drop(columns=['store_address'], inplace=True)

# Present descriptive statistics of numerical columns

In [ ]:
df.describe()

# Identify and Handle Missing Values

In [ ]:
df.isna().sum()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

imputer=ColumnTransformer([('most_frequent',SimpleImputer(missing_values=np.nan, strategy='most_frequent'),['latitude','longitude'])],remainder='passthrough',verbose_feature_names_out=False).set_output(transform='pandas')
df=imputer.fit_transform(df)
test_df=imputer.fit_transform(test_df)

In [ ]:
df.isna().sum()

In [ ]:
test_df.isna().sum()

# Identify and handle duplicates

In [ ]:
init_samples = df.shape[0]
df.drop_duplicates(inplace=True, ignore_index=True)
print(f"Duplicates removed: {init_samples - df.shape[0]}")

# Identify and handle outliers

In [ ]:
outliers_present = {'Train': {}, 'Test': {}}

IQR_multiplier = 2
for col in ['latitude', 'longitude', 'rating_count', 'hours_ago']:
  Q1 = df[col].quantile(0.25)
  Q3 = df[col].quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - IQR_multiplier * IQR
  upper_bound = Q3 + IQR_multiplier * IQR

  for i in (df, test_df):
    outliers = i[(i[col] < lower_bound) | (i[col] > upper_bound)].shape[0]
    if i.shape[0] > 20000:
      outliers_present['Train'][col] = outliers
    else:
      outliers_present['Test'][col] = outliers
    i[col].clip(lower=lower_bound, upper=upper_bound, inplace=True)

for i in outliers_present:
  print(f"Outliers Removed in {i} Data:")
  for col in outliers_present[i]:
    print(f"    {col}: {outliers_present[i][col]}")
  print()

# Visualizations and Insights

In [ ]:
plot_df = df['rating'].value_counts()

plt.bar(plot_df.index, plot_df)
plt.title("Rating Distribution")
plt.xlabel("Rating")
plt.ylabel("Count")
plt.show()

In [ ]:
plt.hexbin(df['longitude'], df['latitude'], gridsize=40, cmap='Blues')
plt.colorbar(label='Count')
plt.title("Location Density")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

In [ ]:
plt.boxplot(df['rating_count'])
plt.title("Rating Count Distribution")
plt.ylabel("Rating Count")
plt.show()

# Scale Numerical features and Encode Categorical features

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2', device='cpu')

In [ ]:
for i in ('df', 'test_df'):
  temp_df = globals()[i]

  review_embeddings = model.encode(
    temp_df['review'],
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=True,
  )

  temp_df.drop(columns=['review'], inplace=True)
  review_embeddings_df = pd.DataFrame(
      review_embeddings,
      columns=[f"review_{i}" for i in range(0, review_embeddings.shape[1])]
  )
  temp_df = pd.concat([temp_df, review_embeddings_df], axis=1)

  globals()[i] = temp_df

In [ ]:
del temp_df, review_embeddings, review_embeddings_df

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(
    df.drop(columns=['rating']),
    df['rating'],
    test_size=0.2,
    random_state=42,
)

In [ ]:
num_cols = ['latitude', 'longitude', 'rating_count', 'hours_ago']
pass_cols = train_x.columns[4:].tolist()

In [ ]:
transformer = ColumnTransformer(transformers=[
    ('ss', StandardScaler(), num_cols),
    ('pass', 'passthrough', pass_cols)
])
transformer.fit(train_x, train_y)

In [ ]:
for var in ('train_x', 'valid_x'):
    i = globals()[var]

    transformed = transformer.transform(i)
    feature_names = transformer.get_feature_names_out()
    index = i.index
    i = pd.DataFrame(transformed, columns=feature_names, index=index)

    globals()[var] = i

# Model Building (at least 7)

In [ ]:
results = {}
models = {
  "Logistic Regression": LogisticRegression(random_state=42, n_jobs=-1, multi_class='multinomial'),
  "Ridge Classifier": RidgeClassifier(random_state=42),
  "PassiveAggressiveClassifier": PassiveAggressiveClassifier(random_state=42),
  "Perceptron": Perceptron(random_state=42, n_jobs=-1),
  "Extra Trees": ExtraTreesClassifier(random_state=42, n_jobs=-1),
  "XGBoost": XGBClassifier(random_state=42, n_jobs=-1),
  "KNN": KNeighborsClassifier(n_jobs=-1)
}

In [ ]:
for model_name in models:
  init_time = time.time()

  try:
    print(f"---Training {model_name}---")
    if model_name == "XGBoost":
      models[model_name].fit(train_x, train_y - 1)
    else:
      models[model_name].fit(train_x, train_y)

    if model_name == "XGBoost":
      valid_pred = models[model_name].predict(valid_x) + 1
    else:
      valid_pred = models[model_name].predict(valid_x)

    f1 = f1_score(valid_y, valid_pred, average='macro')
    accuracy = accuracy_score(valid_y, valid_pred)
    precision = precision_score(valid_y, valid_pred, average='macro')
    recall = recall_score(valid_y, valid_pred, average='macro')
    time_taken = time.time() - init_time

    results[model_name] = {
      "F1": round(f1, 4),
      "Accuracy": round(accuracy, 4),
      "Precision": round(precision, 4),
      "Recall": round(recall, 4),
      "Time": round(time_taken, 2),
    }
    print('\n'.join([f'{metric}: {value}' for metric, value in results[model_name].items()]))
  except Exception as e:
    print(f"{model_name} training failed with the following error:")
    print(e)
    continue
  finally:
    print('\n')

# Comparison of model performances

In [ ]:
results_df = pd.DataFrame(results).T.sort_values(by='Accuracy', ascending=False)
results_df

# Hyperparameter Tuning on any 3 of the models

In [ ]:
df_y = df['rating'].copy()
df.drop(columns=['rating'], inplace=True)
df_x = df
del df

In [ ]:
transformer.fit(df_x, df_y)
for var in ('df_x', 'test_df'):
    i = globals()[var]

    transformed = transformer.transform(i)
    feature_names = transformer.get_feature_names_out()
    i = pd.DataFrame(transformed, columns=feature_names)

    globals()[var] = i

In [ ]:
params = {
    "Ridge Classifier": {
        "alpha": [5],
        "fit_intercept": [True],
        "max_iter": [100],
    },

    "Perceptron": {
        "alpha": [0.005],
        "penalty": ['l2'],
        "max_iter": [100],
        "eta0": [0.01],
        "fit_intercept": [True],
    },

    "XGBoost": {
        "n_estimators": [495],
        "max_depth": [8],
        "learning_rate": [0.12],
        "subsample": [0.81],
        "colsample_bytree": [0.66],
        "gamma": [1.46],
        "reg_alpha": [0.79],
        "reg_lambda": [0.2],
    }
}

In [ ]:
for model_name in params:
  init_time = time.time()
  print(f"Tuning {model_name}")

  search = RandomizedSearchCV(
      estimator=models[model_name],
      param_distributions=params[model_name],
      n_iter=1,
      scoring='accuracy',
      cv=2,
      random_state=42,
      n_jobs=-1
  )

  if model_name == "XGBoost":
    search.fit(df_x, df_y - 1)
  else:
    search.fit(df_x, df_y)

  models[model_name + " (tuned)"] = search.best_estimator_

  if model_name == "XGBoost":
    valid_pred = search.best_estimator_.predict(valid_x) + 1
  else:
    valid_pred = search.best_estimator_.predict(valid_x)

  f1 = f1_score(valid_y, valid_pred, average='macro')
  accuracy = search.best_score_
  precision = precision_score(valid_y, valid_pred, average='macro')
  recall = recall_score(valid_y, valid_pred, average='macro')
  time_taken = round(time.time() - init_time, 2)

  results[model_name + " (tuned)"] = {
      'F1': round(f1, 4),
      'Accuracy': round(accuracy, 4),
      'Precision': round(precision, 4),
      'Recall': round(recall, 4),
      'Time': time_taken,
  }
  print(f"Best score for {model_name}: {search.best_score_:.4f}")
  print(f"Best params for {model_name}: {search.best_params_}")

In [ ]:
results_df = pd.DataFrame(results).T.sort_values(by='Accuracy', ascending=False)
results_df

In [ ]:
best_model = models["XGBoost (tuned)"]
best_model
best_model.fit(df_x, df_y - 1)

In [ ]:
test_pred = best_model.predict(test_df) + 1

submission_df = pd.DataFrame({
    'id': range(len(test_pred)),
    'rating': test_pred
})
submission_df.to_csv('submission.csv', index=False)